In [126]:
import os
import pandas as pd
import numpy as np
import re
from IPython.display import display

# Define the file path
file_path = '../fakedata/g24002.csv'

def get_course_weights(course):
    # Load the course data
    df = pd.read_csv('../fakedata/compcourse.csv')

    # Filter the data for the specified course
    df = df[df['CourseID'] == course]
    as_names = df['Assessment']

    assessment_keywords = {
        'final_exam': ['final exam','oral exam'],
        'midsem_exam': ['midsem exam'],
        'assignment': ['assignment: assignment','assignment: assessment','assignment'],
        'quiz': ['quiz: quiz'],
        'attendance': ['attendance'],
        'lab': ['lab'],
    }

    # Initialize a dictionary to store the headers for each category
    assessment_weights = {assessment_keyword: [] for assessment_keyword in assessment_keywords}

    # Classify headers
    for as_name in as_names:
        for assessment, keywords in assessment_keywords.items():
            if any(keyword in as_name.lower() for keyword in keywords):
                percentage = df[df['Assessment'] == as_name]['Percentage'].values[0]
                assessment_weights[assessment].append(percentage)
                break
        else:
            assessment_weights.setdefault('other', []).append(as_name)

    return assessment_weights

def filter_headers(file_path):  
    # File path
    file_path = '../fakedata/g24002.csv'

    # Extract the numerical part (courseID) using a regular expression
    course = re.search(r'\d+', file_path)
    if course:
        course = int(course.group()[:4])

    # Check if the directory exists
    if os.path.exists(os.path.dirname(file_path)):
        # Load the data
        df = pd.read_csv(file_path)

        # Replace '-' with NaN
        df = df.replace('-', np.nan)

        # Drop columns where all values are NaN
        df = df.dropna(axis=1, how='all')

        # Get the column headers
        headers = df.columns.tolist()

    else:
        print(f"The directory does not exist: {os.path.dirname(file_path)}")
    # Define categories and their keywords
    categories_keywords = {
        'uni_id': ['uni', 'id'],
        'total': ['total'],
        'px_exam': ['da/px exam'],
        'final_exam': ['final exam','oral exam'],
        'midsem_exam': ['midsem exam'],
        'assignment': ['assignment: assignment','assignment: assessment','assignment'],
        'quiz': ['quiz: quiz'],
        'attendance': ['attendance'],
        'lab': ['lab'],
    }

    # Initialize a dictionary to store the headers for each category
    category_headers = {category_keyword: [] for category_keyword in categories_keywords}

    # Classify headers
    for header in headers:
        for category, keywords in categories_keywords.items():
            if any(keyword in header.lower() for keyword in keywords):
                category_headers[category].append(header)
                break
        else:
            category_headers.setdefault('other', []).append(header)

    # If there are multiple attendance columns, keep only the first one
    if len(category_headers['attendance']) > 1:
        category_headers['attendance'] = [category_headers['attendance'][0]]

    # Keep only the columns that fall under the categories 'uni_id', 'exam', 'assignment', 'quiz', 'attendance', and 'lab'
    keep_categories = ['uni_id', 'px_exam', 'final_exam', 'midsem_exam', 'assignment', 'quiz', 'attendance', 'lab']
    keep_headers = [header for category in keep_categories for header in category_headers[category]]
    df = df[keep_headers]

    # Replace NaN values with 0
    df = df.fillna(0)
    # Combine the 'px_exam' and 'final_exam' columns
    if len(category_headers['px_exam']) > 0 and len(category_headers['final_exam']) > 0:
        df[category_headers['final_exam'][0]] = df[category_headers['px_exam'][0]].combine(df[category_headers['final_exam'][0]],
                                                lambda px_exam, final_exam: final_exam if px_exam == 0 else px_exam)
        # Drop the original 'px_exam' and 'final_exam' columns
        df = df.drop(columns=[category_headers['px_exam'][0]])

    return df, category_headers, get_course_weights(course)

############################################

df,category_headers,weights =filter_headers(file_path)
# Print the headers in each category
for category, headers in category_headers.items():
    print(f"{category.capitalize()} headers: {headers}")
    # Reset all display options to their default settings
for asg, weights in weights.items():
    print(f"{asg.capitalize()} weights: {weights}")
    # Reset all display options to their default settings
display(df)

Uni_id headers: ['Uni ID']
Total headers: ['Total marks for quizzes (Real)', 'Course total (Real)']
Px_exam headers: ['Quiz: DA/PX Exam (Real)']
Final_exam headers: ['Final Exam (Real)']
Midsem_exam headers: []
Assignment headers: ['Assignment: Assignment 1 (SQL): Marks and Feedback (Real)', 'Assignment: Assignment 2 (Database Theory): Marks and Feedback (Real)']
Quiz headers: ['Quiz: Quiz for Week 2 (deadline: 11:59pm, Aug 1, Tuesday) (Real)', 'Quiz: Quiz for Week 3 (deadline 23:59, August 8, Tuesday) (Real)', 'Quiz: Quiz for Week 4 (deadline: 11:59pm, August 15, Tuesday) (Real)', 'Quiz: Quiz for Week 5 (deadline: 11:59pm, 22 August, Tuesday) (Real)', 'Quiz: Quiz for Week 6 (deadline: 11:59pm, 29 August, Tuesday) (Real)', 'Quiz: Quiz for Week 7 (deadline: 11:59pm, 19 September, Tuesday) (Real)', 'Quiz: Quiz for Week 8 (deadline 11:59pm, 26 September, Tuesday) (Real)', 'Quiz: Quiz for Week 9 (deadline: 23:59, 3 October, Tuesday) (Real)', 'Quiz: Quiz for Week 10 (deadline: 11:59pm, Octo

,Uni ID,Final Exam (Real),Assignment: Assignment 1 (SQL): Marks and Feedback (Real),Assignment: Assignment 2 (Database Theory): Marks and Feedback (Real),"Quiz: Quiz for Week 2 (deadline: 11:59pm, Aug 1, Tuesday) (Real)","Quiz: Quiz for Week 3 (deadline 23:59, August 8, Tuesday) (Real)","Quiz: Quiz for Week 4 (deadline: 11:59pm, August 15, Tuesday) (Real)","Quiz: Quiz for Week 5 (deadline: 11:59pm, 22 August, Tuesday) (Real)","Quiz: Quiz for Week 6 (deadline: 11:59pm, 29 August, Tuesday) (Real)","Quiz: Quiz for Week 7 (deadline: 11:59pm, 19 September, Tuesday) (Real)","Quiz: Quiz for Week 8 (deadline 11:59pm, 26 September, Tuesday) (Real)","Quiz: Quiz for Week 9 (deadline: 23:59, 3 October, Tuesday) (Real)","Quiz: Quiz for Week 10 (deadline: 11:59pm, October 10, Tuesday) (Real)","Quiz: Quiz for Week 11 (deadline: 11:59pm, 17 October, Tuesday) (Real)",Attendance: Lab Engagement (Real),Lab activities (Lab 1-8) (Real)
0,u0000001,31.7,15.7,8.3,0.4,0.6,0.4,0.1,0.5,0.4,0,0.4,0.4,0.5,55.2,2.0
1,u0000002,49.9,13.9,9.6,0.1,0.6,0.6,0.6,0.5,0.3,0.4,0.5,0.6,0.6,95.2,2.2
2,u0000003,37.4,18.2,12.2,0.4,0.6,0.4,0.5,0.5,0.4,0.5,0.5,0.3,0.5,88.7,2.3
3,u0000004,40.2,21.2,13.4,0.3,0.6,0.5,0.4,0.6,0.4,0.6,0.6,0.6,0.5,75.9,2.2
4,u0000005,33.7,11.1,10.8,0.6,0.6,0.5,0.4,0.5,0.6,0,0.4,0,0,38.9,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,u0000689,46.7,21,13,0.5,0.5,0.6,0.6,0.6,0.5,0.6,0.4,0.3,0,71.6,2.2
689,u0000690,25.9,18,5.6,0,0.3,0.5,0.1,0.4,0,0,0.3,0,0.4,99.2,2.1
690,u0000691,48.8,14.3,9.2,0.6,0.5,0.4,0.5,0.6,0.4,0.5,0.6,0.4,0.6,96.5,2.3
691,u0000692,36.7,13.8,9.7,0.4,0.3,0.3,0.2,0.4,0.3,0.3,0.2,0.2,0.4,53.6,2.2
